## Loans for petroleum products
- Drivers obtain loans from microfinance institutions for fueling subsidie
- As a risk analyst in the loan business department, you need to analyze the historical loan records and try to use this data set to train a decision tree model

### Import the libraries of data analysis

In [1]:
import pandas as pd
import numpy as np
import os
os.environ["PATH"]+=os.pathsep+"C:/Program Files/Graphviz 2.44.1/bin"

### Import the data

In [2]:
data = pd.read_excel("oil_data_for_tree.xlsx")
data.head()

KeyboardInterrupt: 

### Data preprocess
- original_list does not need to make special transformation
- aggregate_list is a list containing aggregated numeric vaviables
- distinct_list is a list containing the distinct and counted text vaviable

In [ ]:
original_list=["uid","oil_actv_dt","create_dt","class_new","bad_ind"]
aggregate_list=["oil_amount","discount_amount","sale_amount","amount","pay_amount","coupon_amount","payment_coupon_amount"]
distinct_list=["channel_code","oil_code","scene","source_app","call_source"]

### Data reorganization

In [ ]:
df=data[original_list].copy()
df.head()

In [ ]:
df[aggregate_list]=data[aggregate_list].copy()
df.head()

In [ ]:
df[distinct_list]=data[distinct_list].copy()
df.head()

#### Check for NAN values

In [ ]:
df.isna().sum()

#### Descriptive statistics

In [ ]:
df.describe()

- The probability distribution of data changes with time, and we cannot use all historical data for modeling.
- We better use data from the last six months
- We need to complete the date data and intercept the last six months of data for analysis

##### The loan date and account activation date complement each other

In [ ]:
def time_isna(x,y):
    if str(x) == "NaT":
        x=y
    else:
        x=y
    return x

In [ ]:
df2=df.sort_values(["uid","create_dt"],ascending=False)
df2.head()

In [ ]:
df2["create_dt"] = df2.apply(lambda x: time_isna(x.create_dt,x.oil_actv_dt),axis=1)
df2.head()

- we select data from 3 months(180 days) 

In [ ]:
df2["dtn"]=(df2.oil_actv_dt-df2.create_dt).apply(lambda x :x.days)
df2.head()

In [ ]:
df3 = df2[df2["dtn"]<180]
df3.head()

In [ ]:
len(df3)

##### Find the maximum interval of the historical loan time of the variable "org_list", and eliminate the repetition

In [ ]:
base=df3[original_list]
base.head()

In [ ]:
base["dtn"]=df3["dtn"]
base.head()

In [ ]:
base=base.sort_values(["uid","create_dt"],ascending=False)
base.head()

In [ ]:
base=base.drop_duplicates(["uid"],keep="first")
base.head()

In [ ]:
base.shape

##### Variable derivation of aggregate list

In [ ]:
gn = pd.DataFrame()

In [ ]:
aggregate_list

In [ ]:
tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:len(df3["oil_amount"])).reset_index())

In [ ]:
tp.columns = ['uid',"oil_amount" + '_cnt']

In [ ]:
tp.head()

In [ ]:
for i in aggregate_list:
    # Record number of loans,……， per borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:len(df3[i])).reset_index())
    tp.columns = ['uid',i + '_cnt']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left")
    # Real number of loans,……， per borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.where(df3[i]>0,1,0).sum()).reset_index())
    tp.columns = ['uid',i + '_cnt']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left")
        gn = pd.merge(gn,tp,on = "uid",how = "left")
    # When there are NAN values, the sum of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nansum(df3[i])).reset_index())
    tp.columns = ['uid',i + '_tot']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left") 
    # When there are NAN values, the mean of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanmean(df3[i])).reset_index())
    tp.columns = ['uid',i + '_avg']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left")   
    # When there are NAN values, the maximum of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanmax(df3[i])).reset_index())
    tp.columns = ['uid',i + '_max']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left")   
    # When there are NAN values, the minimum of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanmin(df3[i])).reset_index())
    tp.columns = ['uid',i + '_min']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left") 
    # When there are NAN values, the variance of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanvar(df3[i])).reset_index())
    tp.columns = ['uid',i + '_var']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left")   
    # When there are NAN values, the range of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanmax(df3[i])-np.nanmin(df3[i])).reset_index())
    tp.columns = ['uid',i + '_range']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left") 
    # When there are NAN values, the coefficient of variation of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:np.nanmax(df3[i])-np.nanmin(df3[i])).reset_index())
    tp.columns = ['uid',i + '_cv']
    if gn.empty == True:
        gn = tp
    else:
        gn = pd.merge(gn,tp,on = "uid",how = "left") 

In [ ]:
gn.head()

##### Variable derivation of distinct_list

In [ ]:
gc = pd.DataFrame()
for i in distinct_list:
        # When there are NAN values, the number of loans for each borrower
    tp = pd.DataFrame(df3.groupby("uid").apply(lambda df3:len(set(df3[i]))).reset_index())
    tp.columns = ['uid',i + '_distinct']
    if gc.empty == True:
        gc = tp
    else:
        gc = pd.merge(gc,tp,on = "uid",how = "left") 

In [ ]:
gc.head()

#### To combine variables ：
- original_list 
- aggregate_list
- distinct_list

In [ ]:
fn = pd.merge(base,gn,on = "uid")
fn.head()

In [ ]:
fn = pd.merge(fn,gc,on = "uid")
fn.head()

In [ ]:
fn.shape

In [ ]:
fn=fn.fillna(0)

In [ ]:
fn.head(50)

### Training a decision tree model

In [ ]:
x = fn.drop(["uid","oil_actv_dt","create_dt","bad_ind","class_new"],axis = 1)
x.head()

In [ ]:
y = fn.bad_ind.copy()
y.head()

In [ ]:
from sklearn import tree

In [ ]:
dtree=tree.DecisionTreeRegressor(max_depth = 2,
                                 min_samples_leaf=500,
                                min_samples_split = 5000
                                )

In [ ]:
dtree = dtree.fit(x,y)

#### Output the image of decision tree and make decisions

In [5]:
import pydotplus
from IPython.display import Image
from six import StringIO
import os
os.environ["PATH"]+=os.pathsep+"C:/Program Files/Graphviz 2.44.1/bin"

In [ ]:
dot_data=StringIO()

In [ ]:
tree.export_graphviz(dtree)

In [ ]:
tree.export_graphviz(dtree,
                     out_file = dot_data,
                     class_names=["bad_ind"],
                     filled = True,rounded = True,
                     special_characters = True
                    )

In [ ]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [ ]:
Image(graph.create_png())

##### Actually，value is badrate

In [ ]:
sum(fn.bad_ind)/len(fn.bad_ind)